<a href="https://colab.research.google.com/github/lalit2244/FOOD-WASTE-MANAGEMENT-SYSTEM/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT TITLE: FOOD WASTE MANAGEMENT SYSTEM

# REQUIREMENTS:
gradio
pandas
matplotlib
seaborn
scikit-learn
sqlite3


# APP LINK: https://7d252f32d4e5ea5eeb.gradio.live

## FULL GRADIO APP (DATA+SQL+KPIS+AI ML)

In [ ]:
# =========================
# Install dependencies (Colab / Local)
# =========================
!pip install gradio pandas numpy matplotlib seaborn scikit-learn sqlalchemy --quiet

# Optional but recommended for better models:
# !pip install xgboost --quiet

# =========================
# Full Gradio App
# =========================
import gradio as gr
import pandas as pd
import numpy as np
import sqlite3
import io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
import traceback
import datetime

sns.set_theme()

# -------------------------
# Global state
# -------------------------
db_conn = None  # sqlite3 connection
dataframes = {}  # name -> pandas.DataFrame

# -------------------------
# Helpers
# -------------------------
def read_uploaded_file(file_obj):
    """Safely read a gradio file-like to a pandas DataFrame"""
    if file_obj is None:
        return None
    try:
        # Gradio returns an object with .name in many cases,
        # but sometimes it's already a file-like or path.
        if hasattr(file_obj, "name") and isinstance(file_obj.name, str):
            # Try reading bytes
            return pd.read_csv(file_obj.name)
        else:
            # file_obj likely has .read()
            file_bytes = file_obj.read()
            return pd.read_csv(io.BytesIO(file_bytes))
    except Exception:
        try:
            # last resort: try casting to pandas directly
            return pd.read_csv(file_obj)
        except Exception:
            raise

def ensure_db():
    global db_conn
    if db_conn is None:
        db_conn = sqlite3.connect(":memory:", check_same_thread=False)

def save_table_to_db(name):
    """Save dataframe to SQLite under given name"""
    ensure_db()
    dataframes[name].to_sql(name, db_conn, index=False, if_exists="replace")

def safe_to_markdown(df, n=10):
    if df is None or df.empty:
        return "No results."
    return df.head(n).to_markdown()

# -------------------------
# Upload dataset(s)
# -------------------------
def upload_file(file, table_name):
    try:
        if file is None:
            return "❌ No file uploaded."
        df = read_uploaded_file(file)
        if df is None:
            return "❌ Could not read file."
        # Basic normalization: strip column names
        df.columns = [str(c).strip() for c in df.columns]
        dataframes[table_name] = df
        save_table_to_db(table_name)
        return f"✅ `{table_name}` uploaded: {len(df)} rows, {len(df.columns)} cols.\n\nPreview:\n{safe_to_markdown(df,5)}"
    except Exception as e:
        return f"❌ Error uploading file: {e}\n\n{traceback.format_exc()}"

# -------------------------
# KPI functions
# -------------------------
def general_kpis():
    rows = []
    for name, df in dataframes.items():
        rows.append({
            "table": name,
            "rows": len(df),
            "cols": len(df.columns),
            "missing_values": int(df.isnull().sum().sum())
        })
    if not rows:
        return "No tables loaded."
    return pd.DataFrame(rows).to_markdown(index=False)

def food_waste_kpis():
    # expects tables: providers, receivers, food_listings, claims (if present)
    if 'food_listings' not in dataframes:
        return "Load a `food_listings` table (filename suggested: food_listings_data.csv) to see food-specific KPIs."
    fl = dataframes['food_listings']
    total_items = len(fl)
    total_quantity = int(fl['Quantity'].sum()) if 'Quantity' in fl.columns else "N/A"
    unique_cities = int(fl['Location'].nunique()) if 'Location' in fl.columns else "N/A"
    unique_food_types = int(fl['Food_Type'].nunique()) if 'Food_Type' in fl.columns else "N/A"
    claims_count = len(dataframes['claims']) if 'claims' in dataframes else 0
    completed = 0
    if 'claims' in dataframes and 'Status' in dataframes['claims'].columns:
        completed = int((dataframes['claims']['Status'] == 'Completed').sum())
        completion_rate = f"{(completed/claims_count*100):.1f}%" if claims_count>0 else "N/A"
    else:
        completion_rate = "N/A"
    md = (
        f"- Total food listings: **{total_items}**\n"
        f"- Total quantity: **{total_quantity}**\n"
        f"- Cities covered: **{unique_cities}**\n"
        f"- Food types: **{unique_food_types}**\n"
        f"- Claims: **{claims_count}**, Completed: **{completed}**, Completion Rate: **{completion_rate}**"
    )
    return md

# -------------------------
# SQL runner
# -------------------------
def run_sql(query_text):
    try:
        ensure_db()
        if query_text.strip() == "":
            return "Enter a SQL query."
        df = pd.read_sql_query(query_text, db_conn)
        # also return shape and a small preview
        return f"Rows returned: {len(df)}\n\nPreview:\n{safe_to_markdown(df,10)}"
    except Exception as e:
        return f"SQL Error: {e}\n\n{traceback.format_exc()}"

# Predefined SQL templates for convenience
PREDEFINED_QUERIES = {
    "Show tables": "SELECT name FROM sqlite_master WHERE type='table';",
    "Top 10 food listings by quantity": "SELECT * FROM food_listings ORDER BY Quantity DESC LIMIT 10;",
    "Total quantity by Location": "SELECT Location, SUM(Quantity) as Total_Q FROM food_listings GROUP BY Location ORDER BY Total_Q DESC LIMIT 20;",
    "Claims count by Status": "SELECT Status, COUNT(*) as cnt FROM claims GROUP BY Status;",
    "Providers count by City": "SELECT City, COUNT(*) as Provider_Count FROM providers GROUP BY City ORDER BY Provider_Count DESC;"
}

# -------------------------
# Visualizations
# -------------------------
def plot_bar(table_name, col):
    try:
        if table_name not in dataframes:
            return None
        df = dataframes[table_name]
        if col not in df.columns:
            return None
        vc = df[col].value_counts().iloc[:50]
        fig, ax = plt.subplots(figsize=(8,4))
        sns.barplot(x=vc.values, y=vc.index, ax=ax)
        ax.set_title(f"Top values of {col}")
        plt.tight_layout()
        return fig
    except Exception:
        return None

def plot_pie(table_name, col):
    try:
        if table_name not in dataframes:
            return None
        df = dataframes[table_name]
        if col not in df.columns:
            return None
        vc = df[col].value_counts().iloc[:10]
        fig, ax = plt.subplots(figsize=(6,6))
        ax.pie(vc.values, labels=vc.index, autopct="%1.1f%%", startangle=140)
        ax.set_title(f"Distribution of {col}")
        return fig
    except Exception:
        return None

def plot_heatmap_foodavailability():
    # heatmap of Location vs Food_Type sum(Quantity)
    if 'food_listings' not in dataframes:
        return None
    fl = dataframes['food_listings']
    if not set(['Location','Food_Type','Quantity']).issubset(fl.columns):
        return None
    pivot = fl.pivot_table(index='Location', columns='Food_Type', values='Quantity', aggfunc='sum', fill_value=0)
    fig, ax = plt.subplots(figsize=(10,6))
    sns.heatmap(np.log1p(pivot), cmap='viridis', ax=ax)  # log scale for better visualization
    ax.set_title("Log(Quantity+1) Heatmap: Location vs Food_Type")
    plt.tight_layout()
    return fig

def plot_claims_trend():
    if 'claims' not in dataframes:
        return None
    c = dataframes['claims'].copy()
    if 'Timestamp' not in c.columns:
        return None
    # try to parse Timestamp
    try:
        c['ts'] = pd.to_datetime(c['Timestamp'])
    except:
        # attempt common formats
        c['ts'] = pd.to_datetime(c['Timestamp'], errors='coerce')
    c = c.dropna(subset=['ts'])
    daily = c.groupby(c['ts'].dt.date).size().reset_index(name='count').sort_values('ts')
    fig, ax = plt.subplots(figsize=(8,3))
    sns.lineplot(x='ts', y='count', data=daily, marker='o', ax=ax)
    ax.set_title("Daily claims trend")
    ax.set_xlabel("Date")
    plt.tight_layout()
    return fig

# -------------------------
# Search & filter food listings (and claim action)
# -------------------------
def search_food(query_text="", food_types=None, locations=None, meal_types=None, provider_types=None,
                qmin=None, qmax=None, limit=50):
    if 'food_listings' not in dataframes:
        return "Load `food_listings` table first."
    fl = dataframes['food_listings'].copy()
    # case-insensitive search in Food_Name
    if query_text:
        fl = fl[fl['Food_Name'].str.contains(query_text, case=False, na=False)]
    if food_types:
        fl = fl[fl['Food_Type'].isin(food_types)]
    if locations:
        fl = fl[fl['Location'].isin(locations)]
    if meal_types:
        fl = fl[fl['Meal_Type'].isin(meal_types)]
    if provider_types:
        fl = fl[fl['Provider_Type'].isin(provider_types)]
    if qmin is not None:
        fl = fl[fl['Quantity'] >= qmin]
    if qmax is not None:
        fl = fl[fl['Quantity'] <= qmax]
    if fl.empty:
        return "No items found with these filters."
    return fl.head(limit).to_markdown()

def claim_food(food_id, receiver_id, receiver_name=""):
    """
    Simulate creating a claim for given Food_ID and Receiver_ID.
    Inserts a row into 'claims' dataframe & SQLite db.
    """
    if 'food_listings' not in dataframes:
        return "Load food_listings first."
    fl = dataframes['food_listings']
    if 'Food_ID' not in fl.columns:
        return "food_listings must have a `Food_ID` column."
    if food_id not in fl['Food_ID'].values:
        return f"Food_ID {food_id} not found."
    # create claims table if not present
    ensure_db()
    now = datetime.datetime.utcnow().isoformat()
    new_row = {
        "Claim_ID": int(np.random.randint(10**6, 10**7)),
        "Food_ID": int(food_id),
        "Receiver_ID": int(receiver_id) if receiver_id is not None else None,
        "Status": "Pending",
        "Timestamp": now
    }
    if 'claims' in dataframes:
        claims_df = dataframes['claims']
        claims_df = pd.concat([claims_df, pd.DataFrame([new_row])], ignore_index=True)
    else:
        claims_df = pd.DataFrame([new_row])
    dataframes['claims'] = claims_df
    # sync to DB
    save_table_to_db('claims')
    return f"✅ Claim created: Claim_ID {new_row['Claim_ID']} (Status: Pending)"

# -------------------------
# ML: Claim Success Prediction (classification)
# -------------------------
def train_claim_classifier(model_name="RandomForest", test_size=0.2, random_state=42):
    # Needs 'claims' and 'food_listings' tables
    if not set(['claims','food_listings']).issubset(dataframes.keys()):
        return "Need both `claims` and `food_listings` tables to train claim classifier."
    claims = dataframes['claims'].copy()
    fl = dataframes['food_listings'].copy()
    # Merge on Food_ID
    if 'Food_ID' not in claims.columns or 'Food_ID' not in fl.columns:
        return "Ensure 'Food_ID' exists in both tables."
    merged = claims.merge(fl, on='Food_ID', how='left')
    # Need Status column
    if 'Status' not in merged.columns:
        return "Claims table needs a 'Status' column."
    # Only keep rows with Status known
    merged = merged[merged['Status'].notna()].copy()
    if len(merged) < 30:
        return f"Not enough data to train (need >=30 rows, have {len(merged)})."
    # target: Completed vs not
    merged['target'] = (merged['Status'] == 'Completed').astype(int)
    # Build features: Quantity, encode Food_Type, Meal_Type, Location, Provider_Type
    features = []
    X = pd.DataFrame()
    if 'Quantity' in merged.columns:
        X['Quantity'] = merged['Quantity'].fillna(0).astype(float)
        features.append('Quantity')
    # categorical encode using LabelEncoder for each
    for col in ['Food_Type','Meal_Type','Location','Provider_Type']:
        if col in merged.columns:
            le = LabelEncoder()
            X[col] = le.fit_transform(merged[col].astype(str))
            features.append(col)
    y = merged['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=float(test_size), random_state=int(random_state))
    # choose model
    try:
        if model_name == "LogisticRegression":
            model = LogisticRegression(max_iter=1000)
        elif model_name == "RandomForest":
            model = RandomForestClassifier(n_estimators=200, random_state=int(random_state))
        else:
            # try XGBoost if name matches and installed
            try:
                from xgboost import XGBClassifier
                model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=int(random_state))
            except Exception as e:
                return f"Model {model_name} not available (XGBoost missing). Install xgboost or choose another model."
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)
        report = classification_report(y_test, preds, zero_division=0)
        # feature importance if available
        fi_text = ""
        if hasattr(model, "feature_importances_"):
            fi = pd.DataFrame({"feature": X.columns, "importance": model.feature_importances_}).sort_values('importance', ascending=False)
            fi_text = "\nFeature importances:\n" + fi.to_markdown(index=False)
        return f"✅ Trained {model_name} classifier. Accuracy: {acc:.3f}\n\nClassification report:\n{report}\n{fi_text}"
    except Exception as e:
        return f"Training error: {e}\n\n{traceback.format_exc()}"

# -------------------------
# ML: Demand Forecasting (regression)
# -------------------------
def train_demand_model(model_name="RandomForest", test_size=0.2, random_state=42):
    if 'food_listings' not in dataframes:
        return "Load `food_listings` table."
    fl = dataframes['food_listings'].copy()
    # Aggregate by (Location, Food_Type) -> Quantity sum
    if not set(['Location','Food_Type','Quantity']).issubset(fl.columns):
        return "food_listings must have columns: Location, Food_Type, Quantity"
    agg = fl.groupby(['Location','Food_Type'])['Quantity'].sum().reset_index()
    # Encode Location & Food_Type
    le_loc = LabelEncoder()
    le_food = LabelEncoder()
    X = pd.DataFrame({
        'Location_enc': le_loc.fit_transform(agg['Location'].astype(str)),
        'FoodType_enc': le_food.fit_transform(agg['Food_Type'].astype(str))
    })
    y = agg['Quantity'].astype(float)
    if len(X) < 10:
        return "Not enough aggregated examples (need >=10) to train demand model."
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=float(test_size), random_state=int(random_state))
    try:
        if model_name == "LinearRegression":
            model = LinearRegression()
        else:
            model = RandomForestRegressor(n_estimators=200, random_state=int(random_state))
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        mse = mean_squared_error(y_test, preds)
        r2 = r2_score(y_test, preds)
        # keep encoders & model in global for prediction
        global demand_model_state
        demand_model_state = {
            "model": model,
            "le_loc": le_loc,
            "le_food": le_food,
            "agg": agg
        }
        return f"✅ Trained {model_name} demand model.\nMSE: {mse:.2f}, R²: {r2:.3f}\nYou can now predict demand for Location & Food_Type."
    except Exception as e:
        return f"Training error: {e}\n\n{traceback.format_exc()}"

def predict_demand(location, food_type):
    if 'demand_model_state' not in globals():
        return "Train a demand model first."
    state = demand_model_state
    model = state['model']
    le_loc = state['le_loc']
    le_food = state['le_food']
    try:
        loc_enc = le_loc.transform([location])[0] if location in le_loc.classes_ else None
        food_enc = le_food.transform([food_type])[0] if food_type in le_food.classes_ else None
        # If new unseen, approximate by nearest or return message
        if loc_enc is None or food_enc is None:
            return "Location or Food_Type unseen by model. Train with more data including this location/type."
        Xp = np.array([[loc_enc, food_enc]])
        pred = model.predict(Xp)[0]
        return f"📈 Predicted demand (units): {pred:.1f}"
    except Exception as e:
        return f"Prediction error: {e}\n\n{traceback.format_exc()}"

# -------------------------
# Utility: download a table as CSV (return bytes)
# -------------------------
def download_table_csv(table_name):
    if table_name not in dataframes:
        return None
    csv_bytes = dataframes[table_name].to_csv(index=False).encode()
    return io.BytesIO(csv_bytes)

# -------------------------
# Build Gradio UI
# -------------------------
with gr.Blocks() as demo:
    gr.Markdown("# 🍽️ Food Waste Management — Gradio All-in-One")
    gr.Markdown("Upload datasets, run SQL, visualize, search, claim and train ML models — all from this UI. Use the `share=True` link to get a public URL.")

    with gr.Tab("1️⃣ Upload Data"):
        gr.Markdown("Upload CSV files and give them table names (suggested names: providers, receivers, food_listings, claims).")
        with gr.Row():
            file_input = gr.File(label="CSV file")
            name_input = gr.Textbox(label="Table name (e.g. food_listings)", value="mytable")
            upload_btn = gr.Button("Upload to DB")
        upload_output = gr.Textbox(label="Upload status")
        download_name = gr.Textbox(label="Table name to download", value="food_listings")
        download_btn = gr.Button("Download CSV")
        download_file = gr.File(label="Download CSV")
        upload_btn.click(upload_file, inputs=[file_input, name_input], outputs=[upload_output])
        download_btn.click(download_table_csv, inputs=[download_name], outputs=[download_file])

    with gr.Tab("2️⃣ KPIs"):
        gr.Markdown("General KPIs for all loaded tables and food-waste-specific KPIs.")
        kpi_gen_btn = gr.Button("Show General KPIs")
        kpi_gen_out = gr.Textbox()
        kpi_fw_btn = gr.Button("Show Food Waste KPIs")
        kpi_fw_out = gr.Markdown()
        kpi_gen_btn.click(lambda: general_kpis(), inputs=None, outputs=kpi_gen_out)
        kpi_fw_btn.click(lambda: food_waste_kpis(), inputs=None, outputs=kpi_fw_out)

    with gr.Tab("3️⃣ Visualizations"):
        gr.Markdown("Select table & column to plot. Also special heatmap & claims trend.")
        with gr.Row():
            viz_table = gr.Textbox(label="Table name", value="food_listings")
            viz_col = gr.Textbox(label="Column name (for bar/pie)")
        with gr.Row():
            bar_btn = gr.Button("Bar plot")
            pie_btn = gr.Button("Pie chart")
            heat_btn = gr.Button("Food Availability Heatmap")
            trend_btn = gr.Button("Claims trend")
        bar_plot = gr.Plot()
        pie_plot = gr.Plot()
        heat_plot = gr.Plot()
        trend_plot = gr.Plot()
        bar_btn.click(plot_bar, inputs=[viz_table, viz_col], outputs=bar_plot)
        pie_btn.click(plot_pie, inputs=[viz_table, viz_col], outputs=pie_plot)
        heat_btn.click(plot_heatmap_foodavailability, inputs=None, outputs=heat_plot)
        trend_btn.click(plot_claims_trend, inputs=None, outputs=trend_plot)

    with gr.Tab("4️⃣ Search & Claim"):
        gr.Markdown("Search food listings with filters. Use `Claim this` to create a claim row.")
        with gr.Row():
            q_text = gr.Textbox(label="Search text (Food_Name)", value="")
            q_food_type = gr.Textbox(label="Food_Type filter (comma separated)", value="")
            q_location = gr.Textbox(label="Location filter (comma separated)", value="")
        with gr.Row():
            q_meal_type = gr.Textbox(label="Meal_Type filter (comma separated)", value="")
            q_provider_type = gr.Textbox(label="Provider_Type filter (comma separated)", value="")
            q_qmin = gr.Number(label="Min Quantity", value=0)
            q_qmax = gr.Number(label="Max Quantity", value=10**9)
        search_btn = gr.Button("Search")
        search_out = gr.Textbox(label="Search Results (markdown)")
        search_btn.click(
            lambda qt, ft, loc, mt, pt, qmin, qmax: search_food(
                query_text=qt,
                food_types=[s.strip() for s in ft.split(",")] if ft.strip() else None,
                locations=[s.strip() for s in loc.split(",")] if loc.strip() else None,
                meal_types=[s.strip() for s in mt.split(",")] if mt.strip() else None,
                provider_types=[s.strip() for s in pt.split(",")] if pt.strip() else None,
                qmin=int(qmin) if qmin is not None else None,
                qmax=int(qmax) if qmax is not None else None,
                limit=100
            ),
            inputs=[q_text, q_food_type, q_location, q_meal_type, q_provider_type, q_qmin, q_qmax],
            outputs=search_out
        )
        with gr.Row():
            claim_foodid = gr.Number(label="Food_ID to claim (from search results)", value=0)
            claim_receiverid = gr.Number(label="Receiver_ID", value=0)
            claim_btn = gr.Button("Claim this Food")
        claim_out = gr.Textbox()
        claim_btn.click(claim_food, inputs=[claim_foodid, claim_receiverid, gr.Textbox(value="")], outputs=claim_out)

    with gr.Tab("5️⃣ SQL Query"):
        gr.Markdown("Run custom SQL queries against the in-memory SQLite DB. Useful tables: providers, receivers, food_listings, claims.")
        predefined = gr.Dropdown(label="Predefined queries", choices=list(PREDEFINED_QUERIES.keys()), value="Show tables")
        sql_area = gr.Textbox(lines=6, label="SQL Query", value=PREDEFINED_QUERIES["Show tables"])
        run_pre_btn = gr.Button("Load Predefined into SQL box")
        run_sql_btn = gr.Button("Run SQL")
        sql_out = gr.Textbox(label="Result / Preview")
        run_pre_btn.click(lambda key: PREDEFINED_QUERIES[key], inputs=[predefined], outputs=[sql_area])
        run_sql_btn.click(run_sql, inputs=[sql_area], outputs=[sql_out])

    with gr.Tab("6️⃣ AI/ML"):
        gr.Markdown("Train & evaluate ML models for Claim Success (classification) and Demand Forecasting (regression).")
        with gr.Row():
            clf_model = gr.Dropdown(label="Classifier", choices=["RandomForest","LogisticRegression","XGBoost"], value="RandomForest")
            clf_btn = gr.Button("Train Claim Classifier")
        clf_out = gr.Textbox()
        clf_btn.click(train_claim_classifier, inputs=[clf_model], outputs=[clf_out])

        with gr.Row():
            reg_model = gr.Dropdown(label="Regressor", choices=["RandomForest","LinearRegression"], value="RandomForest")
            reg_btn = gr.Button("Train Demand Model")
        reg_out = gr.Textbox()
        reg_btn.click(train_demand_model, inputs=[reg_model], outputs=[reg_out])

        gr.Markdown("Predict demand for a specific Location & Food_Type after training demand model.")
        with gr.Row():
            pred_loc = gr.Textbox(label="Location", value="")
            pred_food = gr.Textbox(label="Food_Type", value="")
            pred_btn = gr.Button("Predict Demand")
        pred_out = gr.Textbox()
        pred_btn.click(predict_demand, inputs=[pred_loc, pred_food], outputs=[pred_out])

    with gr.Tab("🔧 Debug / Tables"):
        gr.Markdown("Quick view of loaded tables.")
        table_name_view = gr.Textbox(label="Table name", value="food_listings")
        view_btn = gr.Button("Show head")
        view_out = gr.Dataframe()
        view_btn.click(lambda t: dataframes[t].head(50) if t in dataframes else pd.DataFrame(), inputs=[table_name_view], outputs=[view_out])

    with gr.Row():
        gr.Markdown("----")
        gr.Markdown("App created by your assistant. Use `share=True` to get a public URL when launched.")

# Launch app with public link
demo.launch(share=True, server_name="0.0.0.0")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d252f32d4e5ea5eeb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
